# From Messy Data to Medical Insights: Creating KGs for Drug Repurposing
### Authors: Matilde Pato, Ana Carolina Pereira and Nuno Datia
### [womenENcourage2025](https://womencourage.acm.org/2025/), 18th September
---

## Agenda:

<ul>
  <li><a href="#1">1. Data retrieval and cleaning</a>
    <ul>
      <li><a href="#1.1">1.1. Import libraries</a></li>
      <li><a href="#1.2">1.2. Retrieve DailyMed, Purple Book and Orange Book</a></li>
      <li><a href="#1.3">1.3. Exploring the datasets</a></li>
      <li><a href="#1.4">1.4. Convert all files to JSON using jsonifyer package</a></li>
      <li><a href="#1.5">1.5. Text preprocessing</a></li>
    </ul>
  <li><a href="#2">2. Named Entity Recognition (NER) + Named Entity Linking (NEL)</a></li>
    <ul>
      <li><a href="#2.1">2.1. Import libraries</a></li>
      <li><a href="#2.2">2.2. Configure MER</a></li>
      <li><a href="#2.3">2.3. Extract the entities in a sample</a></li>
      <li><a href="#2.5">2.4. Other ontology</a></li>
    </ul>
</ul>
<ul>
   <li><a href="#3">3. Knowledge Graph</a></li>
   <ul>
      <li><a href="#3.1">3.1. Import Libraries</a></li>
      <li><a href="#3.2">3.2. Enter your Neo4j credentials </a></li>
      <li><a href="#3.3">3.3. Neo4j Handler class</a></li>
      <li><a href="#3.4">3.4. Utility functions for data processing </a></li>
      <li><a href="#3.5">3.5. Establish connection to Neo4j </a></li>
      <li><a href="#3.6">3.6. Import data from JSON files </a></li>
      <li><a href="#3.7">3.7. Close Neo4j connection </a></li>
    </ul>
</ul>


<a id="1"></a>

## 1. Data Retrieval and Text Pre-Processing

**Goal #1**: To retrieve the
1. [DailyMed](https://dailymed.nlm.nih.gov/dailymed/) database contains labeling submitted to the Food and Drug Administration (FDA) by companies for prescription drugs and biological products for human use. We will select a sample of complete drug labels (indications, contraindications, warningsandprecautions, ingredients) to build a knowledge graph.
2. [Purple Book](https://purplebooksearch.fda.gov/) database contains information about all FDA-licensed biological products regulated by the CDER, and FDA-licensed allergenic, cellular and gene therapy, hematologic, and vaccine products regulated by CBER.
3. [Orange Book](https://www.fda.gov/drugs/drug-approvals-and-databases/approved-drug-products-therapeutic-equivalence-evaluations-orange-book) (Approved Drug Products With Therapeutic Equivalence Evaluations) is composed of approved prescription drug products with therapeutic equivalence evaluations (Prescription Drug Product List) and others.

To convert different types of files (XML, CSV, TXT) into JSON format, we are going to apply the [JSONIFYER](https://pypi.org/project/jsonifyer/) tool.

**Goal #2**: **Text preprocessing** constitutes a fundamental stage in [Natural Language Processing](https://en.wikipedia.org/wiki/Natural_language_processing) (NLP), wherein raw textual data is systematically cleaned and transformed into a structured format suitable for computational analysis and machine learning applications. This process improves data quality and analytical utility by eliminating noise, standardizing linguistic forms, and reducing textual complexity.

<a id="1.1"></a>
### 1.1 Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import json
import string
import time
from pathlib import Path

import xml.etree.ElementTree as ET

In [ ]:
# Install the jsonifyer package
!pip install jsonifyer

In [ ]:
# Import specific conversion functions from jsonifyer package
# These functions will help us convert different file formats into JSON:
from jsonifyer import (
    convert_txt,    # For plain text file conversion
    convert_csv,    # For comma-separated values file conversion
    convert_xml     # For XML markup file conversion
)

<a id="1.2"></a>
### 1.2. Retrieve DailyMed, Purple Book and Orange Book

DailyMed is a large dataset that contains **153,971** labels submitted to the FDA, so in this tutorial, we are going to use a **smaller version of the dataset**.

This version is located under the directory "drugs_small".

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change working directory
os.chdir("/content/drive/MyDrive/Colab Notebooks/Tutorial WomenEncourage2025/")
# Define the base directory for our project
BASE_DIR = Path(os.getcwd())

# Set up input and output folder paths
input_folder = BASE_DIR / 'drugs_small'
output_folder = BASE_DIR / 'json'

# Create output directories for each file type
# This mirrors the input structure: xml_files, csv_files, txt_files
file_types = ['xml_files', 'csv_files', 'txt_files']

print("Creating output directory structure...")
for dir_type in file_types:
    output_dir = output_folder / dir_type
    os.makedirs(output_dir, exist_ok=True)
    print(f"✓ Created directory: {output_dir}")

# Create tracking files to keep record of processed files
# These files prevent duplicate processing of the same data
repeated_files = {
    'xml_files': BASE_DIR / 'xml_processed.txt',
    'csv_files': BASE_DIR / 'csv_processed.txt',
    'txt_files': BASE_DIR / 'txt_processed.txt'
}

print("Creating tracking files...")
for file_type, file_path in repeated_files.items():
    if not file_path.exists():
        file_path.touch()  # Create empty file
        print(f"✓ Created tracking file: {file_path}")
    else:
        print(f"ℹ Tracking file already exists: {file_path}")

<a id="1.3"></a>

### 1.3. Exploring the datasets

In [ ]:
# DailyMed
# Read and parse XML
xml_file_path = Path(input_folder) / "xml_files/0d1f10eb-de2e-49c3-bbea-2cafbafe05b1.xml"
tree = ET.parse(xml_file_path)
root = tree.getroot()

print(f"📋 ROOT ELEMENT: <{root.tag}>")
print(f"Root attributes: {root.attrib}")
print("-" * 50)

element_attrs = {}
for elem in root.iter():
    tag = elem.tag
    if tag not in element_attrs:
        element_attrs[tag] = set()
    element_attrs[tag].update(elem.attrib.keys())
for tag, attrs in element_attrs.items():
    if attrs:
        print(f"{tag}: {', '.join(attrs)}")
    else:
        print(f"{tag}: (no attributes)")

In [ ]:
# Print first few lines of XML for manual inspection
with open(xml_file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

print("\n📖 FIRST 20 LINES OF XML:")
print("-" * 30)
for i, line in enumerate(lines[:20], 1):
    print(f"{i:2d}: {line.rstrip()}")

In [ ]:
# Read the file from Purple book database
csv_files = Path(input_folder) / "csv_files/purplebook-search-may-data-download.csv"
purple_book = pd.read_csv(csv_files, sep = ',', quotechar = '"',
                          encoding = 'utf-8', dtype=str)
purple_book.head(4)

In [ ]:
# Read the products.txt file (Orange Book data)
txt_file_path = Path(input_folder) / "txt_files/products.txt"

# Method 1: Manual inspection of raw file content
with open(txt_file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
for i, line in enumerate(lines[:5], 1):
    print(f"{i:2d}: {line.rstrip()}")
# Method 2: Using pandas to read the delimited file
orange_book = pd.read_csv(txt_file_path, sep='~', dtype=str, encoding='utf-8')
print(orange_book.head(4))

<a id="1.4"></a>

### 1.4. Convert all files to JSON using **jsonifyer** package

In [ ]:
# Configure XML processing parameters
print("Configuring XML processing parameters...")
# Define XML namespaces - these help parse XML documents correctly
namespaces = {'': 'urn:hl7-org:v3'}

# Map fields to their XPath locations in the XML
# XPath is a way to navigate XML document structure
field_map = {
    'id': './/id/@root',
    'code.code': './/code/@code',
    'code.codeSystem': './/code/@codeSystem',
    'code.displayName': './/code/@displayName',
    'organization': './/author/assignedEntity/representedOrganization/name',
    'name': './/component/structuredBody/component/section/subject/manufacturedProduct/manufacturedProduct/name',
    'effectiveTime': './/effectiveTime/@value',
    'ingredients.name': './/component/structuredBody/component/section/subject/manufacturedProduct/manufacturedProduct/ingredient/ingredientSubstance/name',
    'ingredients.code': './/component/structuredBody/component/section/subject/manufacturedProduct/manufacturedProduct/ingredient/ingredientSubstance/code/@code',
}

# Define section codes for specific medical document sections
section_codes = {
    'indications': '34067-9',
    'contraindications': '34068-7',
    'warningsAndPrecautions': '34069-5',
    'adverseReactions': '34070-3'
}

# Define paired fields that should be processed together
# This is useful for ingredients that have both names and codes
pairs = {
    'ingredients.name': [
        './/component/structuredBody/component/section/subject/manufacturedProduct/manufacturedProduct/ingredient/ingredientSubstance',
        'name'
    ],
    'ingredients.code': [
        './/component/structuredBody/component/section/subject/manufacturedProduct/manufacturedProduct/ingredient/ingredientSubstance',
        'code/@code'
    ],
}

xml_input_dir = input_folder / 'xml_files'
xml_output_dir = output_folder / 'xml_files'

print("\n" + "-" * 40)
print("PROCESSING XML FILES")
print("-" * 40)
# Call the convert_xml function with all our configured parameters
result = convert_xml(
    directory_path=str(xml_input_dir),
    repeated_path=str(repeated_files['xml_files']),
    repeated_item='name',  # Field to check for duplicates
    output_path=str(xml_output_dir),
    converter="python",  # Use Python XML parser
    field_map=field_map,
    extra_fields=section_codes,
    namespaces=namespaces,
    pairs=pairs,
    root="document"  # Root element of XML documents
)

print("\n" + "-" * 40)
print("COMPLETED XML FILES")
print("-" * 40)
# Count actual files to show real results
xml_files = [f for f in os.listdir(xml_input_dir) if f.endswith('.xml')]
json_files = [f for f in os.listdir(xml_output_dir) if f.endswith('.json')]

total_xml = len(xml_files)
total_json = len(json_files)

print(f"✓ Successfully converted {total_json} out of {total_xml} XML files")
print(f"✓ Success rate: {(total_json/total_xml)*100:.1f}%")

if total_json < total_xml:
    print(f"{total_xml - total_json} files had conversion issues")

In [ ]:
# Process CSV FILES
csv_input_dir = input_folder / 'csv_files'
csv_output_dir = output_folder / 'csv_files'

print("\n" + "-" * 40)
print("PROCESSING CSV FILES")
print("-" * 40)

# Process each CSV file in the directory
for filename in os.listdir(csv_input_dir):
# Skip files that aren't CSV format
  if not filename.lower().endswith('.csv'):
      print(f"ℹ Skipping non-CSV file: {filename}")
      continue

  filepath = csv_input_dir / filename
  print(f"INFO: Processing CSV file: {filename}...")

  # Convert individual CSV file to JSON
  result = convert_csv(
      file_path=str(filepath),
      output_path=str(csv_output_dir),
      repeated_path=str(repeated_files['csv_files']),
      repeated_item='Proper Name',  # Column to check for duplicates
      skiprows=3  # Skip first 3 rows (likely headers/metadata)
  )

  print(f"INFO: Conversion result for {filename}: {result}")

  # Display success message
  if isinstance(result, dict) and 'message' in result:
      print(f"{result['message']}")
  else:
      print(f"{filename} processed successfully")

In [ ]:
# Process TXT files
txt_input_dir = input_folder / 'txt_files'
txt_output_dir = output_folder / 'txt_files'

print("\n" + "-" * 40)
print("PROCESSING TXT FILES")
print("-" * 40)

# Process each text file in the directory
for filename in os.listdir(txt_input_dir):
  # Skip files that aren't text format
  if not filename.lower().endswith('.txt'):
      print(f"Skipping non-TXT file: {filename}")
      continue

  filepath = txt_input_dir / filename
  print(f"INFO: Processing TXT file: {filename}...")

  # Convert delimited text file to JSON
  result = convert_txt(
      file_path=str(filepath),
      output_path=str(txt_output_dir),
      repeated_path=str(repeated_files['txt_files']),
      repeated_item='Ingredient',  # Field to check for duplicates
      delimiter='~'  # Custom delimiter (tilde instead of comma)
  )

  print(f"INFO: Conversion result for {filename}: {result}")

  # Display success message
  if isinstance(result, dict) and 'message' in result:
      print(f"{result['message']}")
  else:
      print(f"{filename} processed successfully")

<a id="1.5"></a>
### 1.5. Text Preprocessing

Raw pharmaceutical data from **DailyMed**, **Purple Book**, and **Orange Book** contains inconsistent formatting, medical abbreviations, and varying drug name spellings that hinder knowledge graph construction. Our preprocessing pipeline standardizes this text to enable accurate drug-disease relationship extraction.

#### Key Processing Steps

1. Encoding fixes: Resolve character encoding issues and HTML entities
2. Medical standardization: Expand abbreviations (mg → milligram) and normalize drug names (paracetamol → acetaminophen)
3. Text cleaning: Remove stopwords, normalize punctuation, and standardize case
4. Field processing: Clean specific pharmaceutical fields (ingredients, indications, contraindications, warnings)

In [ ]:
# Install required packages (run this cell first in Colab)
!pip install nltk

In [ ]:
import nltk

# Download required NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')

Processes one pharmaceutical JSON file

In [ ]:
# Define preprocessing dictionaries
BIO_ABBREVIATIONS = {
    'mg': 'milligram',
    'ml': 'milliliter',
    'iv': 'intravenous',
    'po': 'oral',
    'bid': 'twice daily',
    'tid': 'three times daily',
    'qid': 'four times daily',
    'prn': 'as needed',
    'stat': 'immediately'
}

DRUG_CORRECTIONS = {
    'acetaminophen': 'acetaminophen',
    'paracetamol': 'acetaminophen',
    'tylenol': 'acetaminophen',
    'ibuprofen': 'ibuprofen',
    'advil': 'ibuprofen',
    'motrin': 'ibuprofen'
}

CUSTOM_STOPWORDS = [
    'patient', 'patients', 'medication', 'drug', 'treatment', 'therapy',
    'dose', 'dosage', 'administration', 'clinical', 'study', 'trial'
]

# Fields to process
FIELDS_TO_PROCESS = [
    "ingredients",
    "indications",
    "contraindications",
    "warningsAndPrecautions",
    "adverseReactions"
]

In [ ]:
# Example of a file content
json_file_path = Path(output_folder) / "xml_files/0d1f10eb-de2e-49c3-bbea-2cafbafe05b1.json"

#  Load JSON data
with open(json_file_path) as f:
    drugs_data = json.load(f)

# Process each specified field
for field in FIELDS_TO_PROCESS:
    if field in drugs_data and isinstance(drugs_data[field], str):
        print(f"\nProcessing field: {field}")

        # Get the original text
        text = drugs_data[field]
        print(f"Original text preview: {text[:50]}...")

        # Preserve original text
        drugs_data[f"{field}_original"] = text

        # Step 1: Fix common encoding issues
        encoding_fixes = {
            '\x92': "'", '\x93': '"', '\x94': '"',  # Smart quotes
            '\x96': '-', '\x97': '-', '\xa0': ' ',  # Dashes and spaces
            '&amp;': '&', '&lt;': '<', '&gt;': '>'  # HTML entities
        }

        for old, new in encoding_fixes.items():
            text = text.replace(old, new)

        # Step 2: Convert to lowercase for processing
        text = text.lower()

        # Step 3: Standardize punctuation
        # Normalize multiple dashes to single dash
        text = re.sub(r'-+', '-', text)

        # Add spaces around punctuation for better tokenization
        for punct in [',', '.', ';', ':', '!', '?', '(', ')']:
            text = re.sub(f'\\{punct}', f' {punct} ', text)

        # Normalize whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        # Step 4: Expand biomedical abbreviations
        for abbr, full_form in BIO_ABBREVIATIONS.items():
            # Use word boundaries to avoid partial matches
            pattern = rf'\b{abbr.lower()}\b'
            text = re.sub(pattern, full_form.lower(), text)

        # Step 5: Correct drug name spellings
        words = word_tokenize(text)
        corrected_words = []

        for word in words:
            # Check if word needs spelling correction
            if word in DRUG_CORRECTIONS:
                corrected_words.append(DRUG_CORRECTIONS[word])
                print(f"Corrected: {word} → {DRUG_CORRECTIONS[word]}")
            else:
                corrected_words.append(word)

        text = ' '.join(corrected_words)

        # Step 6: Remove stopwords
        # Get standard English stopwords
        try:
            english_stops = set(stopwords.words('english'))
        except:
            # Fallback if NLTK stopwords not available
            english_stops = {'the', 'a', 'an', 'and', 'or', 'but', 'in',
                             'on', 'at', 'to', 'for', 'of', 'with', 'by'}

        # Combine with custom medical stopwords
        all_stopwords = english_stops.union(set(CUSTOM_STOPWORDS))

        # Tokenize and filter stopwords
        tokens = word_tokenize(text)
        filtered_tokens = [token for token in tokens if token not in
                           all_stopwords and token not in string.punctuation]

        cleaned_text = ' '.join(filtered_tokens)
        print(f"Cleaned text preview: {cleaned_text[:100]}...")

        # Update the field with cleaned text
        drugs_data[field] = cleaned_text

        print(f"✓ Successfully processed: {drugs_data}")

**Extra**: Pipeline for clean ALL data

1. ```clean_biomedical_text(text)```

    Purpose: Cleans raw pharmaceutical text
*   Fixes encoding issues (smart quotes, HTML entities)
*   Expands abbreviations (mg → milligram, iv → intravenous)
*   Standardizes drug names (paracetamol → acetaminophen)
*   Removes stopwords and normalizes text


2. ```process_single_json_file(input_file, output_file)```

    Purpose: Processes one pharmaceutical JSON file
*   Loads drug data from DailyMed/Purple Book format
*   Cleans specific fields: ingredients, indications, contraindications, warnings
*   Preserves original text in *_original fields
*   Saves cleaned version


3. ```process_file_wrapper(file_task)```

    Purpose: Enables parallel processing
*   Wraps single file processing for multiprocessing
*   Handles errors without crashing entire batch
*   Tracks processing time and results
*   Returns success/failure status


4. ```collect_all_json_files(input_dir, output_dir)```

    Purpose: File discovery and organization
*   Finds all JSON files in directory tree
*   Creates matching output directory structure
*   Returns list of file processing tasks
*   Preserves folder organization

***Parallel Processing Flow***
```# Main batch processing function uses all four:
def preprocess_json_task_batch(input_dir, output_dir, batch_size=50):
    # 1. Find all files
    file_tasks = collect_all_json_files(input_dir, output_dir)
    
    # 2. Process in parallel batches
    with ProcessPoolExecutor() as executor:
        # 3. Each worker uses process_file_wrapper
        futures = [executor.submit(process_file_wrapper, task)
                  for task in batch_tasks]
        
        # 4. Which calls process_single_json_file
        # 5. Which calls clean_biomedical_text for each field```

In [ ]:
def clean_biomedical_text(text):
    """Simplified biomedical text cleaning"""
    # Basic encoding fixes
    encoding_fixes = {
        '\x92': "'", '\x93': '"', '\x94': '"', '\x96': '-',
        '\x97': '-', '\xa0': ' ', '&amp;': '&', '&lt;': '<', '&gt;': '>'
    }
    for old, new in encoding_fixes.items():
        text = text.replace(old, new)

    # Lowercase and normalize punctuation
    text = text.lower()
    text = re.sub(r'-+', '-', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Expand abbreviations
    for abbr, full_form in BIO_ABBREVIATIONS.items():
        text = re.sub(rf'\b{abbr}\b', full_form, text)

    # Tokenize and process
    try:
        tokens = word_tokenize(text)
        english_stops = set(stopwords.words('english'))
    except:
        tokens = text.split()
        english_stops = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}

    # Filter tokens
    all_stopwords = english_stops.union(set(CUSTOM_STOPWORDS))
    filtered_tokens = [
        DRUG_CORRECTIONS.get(token, token) for token in tokens
        if token not in all_stopwords and token not in string.punctuation
    ]

    return ' '.join(filtered_tokens)

In [ ]:
def process_single_json_file(input_file, output_file, fields_to_process=None):
    """Process a single JSON file"""
    if fields_to_process is None:
        fields_to_process = FIELDS_TO_PROCESS

    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Process each field
        for field in fields_to_process:
            if field in data and isinstance(data[field], str):
                data[f"{field}_original"] = data[field]
                data[field] = clean_biomedical_text(data[field])

        # Save processed data
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

        return {'status': 'success', 'file': input_file}

    except Exception as e:
        return {'status': 'error', 'file': input_file, 'error': str(e)}

In [ ]:
def process_file_wrapper(file_task):
    """Wrapper for parallel processing"""
    input_file, output_file = file_task
    start_time = time.time()

    result = process_single_json_file(input_file, output_file)
    result['processing_time'] = time.time() - start_time
    result['input_file'] = input_file
    result['output_file'] = output_file

    return result

In [ ]:
def collect_all_json_files(input_dir, output_dir):
    """Collect all JSON files for processing"""
    file_tasks = []

    for root, dirs, files in os.walk(input_dir):
        relative_path = os.path.relpath(root, input_dir)
        output_subdir = output_dir if relative_path == '.' else os.path.join(output_dir, relative_path)
        os.makedirs(output_subdir, exist_ok=True)

        for f in files:
            if f.endswith(".json"):
                input_file = os.path.join(root, f)
                output_file = os.path.join(output_subdir, f)
                file_tasks.append((input_file, output_file))

    return file_tasks

**Preprocessing using Batch Processing**

When processing thousands of DailyMed XML files (like the ~100,000+ drug labels in the full database), loading everything into memory at once would crash most systems. Batch processing solves this by:

1. Memory efficiency: Process files in small chunks (50-100 files at a time)
2. Fault tolerance: One corrupted file won't stop the entire process
3. Progress tracking: Monitor completion in real-time
4. Scalability: Handle datasets from hundreds to hundreds of thousands of files


##### Alternatives:
1. ProcessPoolExecutor: CPU-intensive text processing
2. ThreadPoolExecutor: I/O-heavy file operations

In [ ]:
# Install required packages for parallel processing
!pip install tqdm

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import cpu_count
from tqdm import tqdm
import threading

In [ ]:
def preprocess_json_task_batch(input_dir, output_dir, batch_size=50):
    """
    Process files in batches for optimal parallel processing
    """
    print("=" * 60)
    print("BATCH PARALLEL PREPROCESSING TASK")
    print("=" * 60)

    # Collect all files
    file_tasks = collect_all_json_files(input_dir, output_dir)
    if not file_tasks:
        print("No JSON files found to process!")
        return

    # Process in batches
    total_batches = (len(file_tasks) + batch_size - 1) // batch_size
    print(f"Processing {len(file_tasks)} files in {total_batches} batches of {batch_size}")

    all_successful = []
    all_failed = []

    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, len(file_tasks))
        batch_tasks = file_tasks[start_idx:end_idx]

        print(f"\nProcessing batch {batch_num + 1}/{total_batches} ({len(batch_tasks)} files)")

        # Process current batch
        with ProcessPoolExecutor(max_workers=cpu_count()) as executor:
            futures = [executor.submit(process_file_wrapper, task) for task in batch_tasks]

            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Batch {batch_num + 1}"):
                result = future.result()

                if result['status'] == 'success':
                    all_successful.append(result)
                else:
                    all_failed.append(result)

    # Final results
    print(f"\nTotal successful: {len(all_successful)}")
    print(f"Total failed: {len(all_failed)}")

    if all_failed:
        print("Failed files:")
        for failed in all_failed[:5]:  # Show first 5 failures
            print(f"  - {failed['file']}: {failed.get('error', 'Unknown error')}")

In [ ]:
json_input_dir = output_folder / 'xml_files'
output_dir = output_folder / 'json_clean_files'

# Batch processing for very large datasets
preprocess_json_task_batch(json_input_dir, output_dir, batch_size=100)

<a id="2"></a>

## 2. Named Entity Recognition (NER) + Named Entity Linking (NEL)

**Goal**: To recognize drugs, chemicals, and disease entities in the retrieved datasets and to link them to the respective ontology identifiers.

We are going to use the [Chemical Entities of Biological Interest](https://www.ebi.ac.uk/chebi/) (ChEBI), [Disease Ontology](https://disease-ontology.org/) (DO), and the [Orphanet](https://www.orpha.net/).
To perform NER and NEL, we are going to apply Minimal Named-Entity Recognizer [MER](https://pypi.org/project/merpy/) tool.

<a id="2.1"></a>
### 2.1. Import libraries

In [ ]:
!pip3 install ssmpy

In [ ]:
!pip3 install merpy

In [ ]:
import merpy
import multiprocessing
from collections import Counter

<a id="2.2"></a>
### 2.2. Configure MER


First, we are going to download the **ChEBI** and **Disease Ontology (DO)** ontologies:

**IMPORTANT:**  Only for OSX
see: https://github.com/prisma-labs/python-graphql-client/issues/13

In [ ]:
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
response = urllib.request.urlopen('https://www.python.org')

In [ ]:
# Define the base directory for our project
BASE_DIR = Path(os.getcwd())

# Set up output folder paths
output_folder = BASE_DIR / 'json'

In [ ]:
ontologies_dir = BASE_DIR / "ontologies"
os.makedirs(ontologies_dir, exist_ok=True)
os.chdir(ontologies_dir)

In [ ]:
lexicon_name = 'doid'
try:
    # Try to load existing lexicon
    merpy.get_entities("test", lexicon_name)
    print(f"{lexicon_name} lexicon already exists")
except:
    # Download if doesn't exist or is corrupted
    # Download DO (2025-06-27 version)
    print(f"Downloading {lexicon_name} lexicon...")
    merpy.download_lexicon('http://purl.obolibrary.org/obo/doid.owl',
                          'doid', ltype='owl')
    print(f"{lexicon_name} lexicon downloaded")

In [ ]:
lexicon_name = 'chebi'
try:
    # Try to load existing lexicon
    merpy.get_entities("test", lexicon_name)
    print(f"{lexicon_name} lexicon already exists")
except:
    # Download if doesn't exist or is corrupted
    # Download ChEBI (2021-07-01 version)
    print(f"Downloading {lexicon_name} lexicon...")
    merpy.download_lexicon('http://purl.obolibrary.org/obo/chebi/chebi_lite.owl',
                       'chebi', ltype='owl')
    print(f"{lexicon_name} lexicon downloaded")

In [ ]:
# Install gawk using apt-get (system package manager)
!apt-get update
!apt-get install -y gawk

In [ ]:
# Then, we need to process the downloaded files into lexicons that MER can use
merpy.process_lexicon("doid", ltype="owl")
merpy.process_lexicon("chebi", ltype="owl")

In [ ]:
# We are going to delete obsolete concepts still present in the ontologies file:
merpy.delete_obsolete("chebi")
merpy.delete_obsolete("doid")

In [ ]:
# Let's check the lexicons available for MER:
merpy.show_lexicons()

<a id="2.3"></a>
### 2.3. Extract the entities in a sample

In [ ]:
# Example 1
json_file_path = Path(output_folder) / "xml_files/0d1f10eb-de2e-49c3-bbea-2cafbafe05b1.json"

#  Load JSON data
with open(json_file_path) as f:
    drugs_data = json.load(f)

In [ ]:
# Let's check the contents of the file:
print(drugs_data.keys())

In [ ]:
# We want to recognize the entities present in indications and ingredients.
indications_test = drugs_data["indications"]
merpy.get_entities(indications_test, 'doid')

Let's check if the annotations make sense. For instance, access the link http://purl.obolibrary.org/obo/DOID_1184.

The entity 'nephrotic syndrome' in the file was linked to the DO concept 'nephrotic syndrome' with the identifier 'DOID:1184', which seems correct!

Let's apply MER to recognize chemical entities and to link them to ChEBI concepts:

In [ ]:
merpy.get_entities(indications_test, 'chebi')

Accessing the link http://purl.obolibrary.org/obo/CHEBI_4027, we can see that the entity 'Cyclophosphamide' was linked to the ChEBI concept 'steroid', which has the identifier 'CHEBI:4027'.

In [ ]:
ingredients_test = drugs_data["ingredients"]
merpy.get_entities(ingredients_test, 'chebi')

In [ ]:
ingredients_test = drugs_data["ingredients"]
print(ingredients_test)

In [ ]:
merpy.get_entities(ingredients_test[0]["name"], 'chebi')

<a id="2.4"></a>
### 2.4. Others ontologies

First, we are going to download the [**Orphanet (ORDO)**](https://www.orpha.net/) ontology

In [ ]:
!pip3 install owlready2

In [ ]:
from nltk import pos_tag
from owlready2 import get_ontology
from difflib import SequenceMatcher

In [ ]:
os.chdir(ontologies_dir)

In [ ]:
# Version 4.7 (english) - 30 Jun 2025
try:
    # Try to load existing lexicon
    onto = get_ontology("https://www.orphadata.com/data/ontologies/ordo/last_version/ORDO_en_4.7.owl").load()
    print("Ontology loaded successfully.")
except Exception as e:
    print(f"Error loading ontology: {e}")

In [ ]:
# Build a comprehensive mapping of disease terms to ontology IRIs
disease_terms = {}
synonym_count = 0
label_count = 0

# Iterate through all classes in the ontology
for cls in onto.classes():
    # Add primary labels
    if hasattr(cls, "label") and cls.label:
        for label in cls.label:
            disease_terms[label.lower()] = cls.iri
            label_count += 1

    # Add synonyms of different types
    for prop in ["hasExactSynonym", "hasRelatedSynonym", "hasNarrowSynonym"]:
        if hasattr(cls, prop):
            for synonym in getattr(cls, prop):
                disease_terms[synonym.lower()] = cls.iri
                synonym_count += 1

In [ ]:
# Example 2
sample_text = """
    The patient was diagnosed with Niemann-Pick disease type C.
    There were no signs of cystic fibrosis. However, we cannot rule out
    a mild form of Gaucher disease based on the enzyme analysis.
    """

In [ ]:
# Extract potential disease mentions from text using regular expression patterns.

# Define patterns for disease recognition
disease_patterns = [
    # Common disease suffixes
    r'\b\w+(?:disease|syndrome|disorder|deficiency|infection|malignancy|cancer|leukemia|lymphoma)\b',

    # Diseases with modifiers
    r'(?:acute|chronic|severe|invasive)\s+\w+\s+\w+',

    # Hyphenated disease names
    r'\b\w+-\w+\s+(?:disease|syndrome|disorder|infection)\b',

    # Specific complex disease patterns
    r'\b\w+\s+versus-host\s+disease\b',

    # Explicitly named diseases
    r'\bNiemann-Pick\s+disease\b',
    r'\bGaucher\s+disease\b',
    r'\bFabry\s+disease\b',
    r'\bCystic\s+fibrosis\b',
    r'\bHuntington\'s\s+disease\b',
    r'\bAlzheimer\'s\s+disease\b',
    r'\bParkinson\'s\s+disease\b'
]

# Find all matches for each pattern
matches = []
for pattern in disease_patterns:
    matches.extend(re.findall(pattern, sample_text, re.IGNORECASE))

# Remove duplicates and return
unique_matches = list(set(matches))
print(f"Extracted {len(unique_matches)} potential disease entities")
print(unique_matches)

In [ ]:
similarity_threshold = 0.8

# Process each disease with inline matching
for disease in unique_matches:
    # Inline ontology matching - replaces find_disease_in_ontology function
    search_name = disease.lower().strip()
    ontology_id = None

    # Try exact match first
    if search_name in disease_terms:
        ontology_id = disease_terms[search_name]
        print(f"Exact match: {disease} -> {ontology_id}")
    else:
        # Fuzzy matching with similarity scoring
        best_match_iri = None
        highest_score = 0.0
        best_term = None

        for term, iri in disease_terms.items():
            score = SequenceMatcher(None, search_name, term).ratio()
            if score > highest_score and score > similarity_threshold:
                best_match_iri = iri
                highest_score = score
                best_term = term

        if best_match_iri:
            ontology_id = best_match_iri
            print(f"Fuzzy match: {disease} -> {best_term} -> {ontology_id} (score: {highest_score:.2f})")
        else:
            print(f"No match found for: {disease}")

    # Use ontology_id here for further processing
    if ontology_id:
        # Your processing logic here
        print(f"Processing {disease} with ID: {ontology_id}")
    else:
        print(f"Skipping {disease} - no ontology mapping found")

Next, we are going to iterate over each file present in the sample directory, annotate it, and create the respective entity file

Now we have both the files ('data/json' directory) and the respective entities files ('data/ner' directory), and the next step will be the creation of the **Knowledge Graph**.

<a id="3"></a>
## 3. Knowledge Graph

**Goal**: Build a pharmaceutical Knowledge Graph linking drugs, diseases, administration routes, and approval years. Using normalized JSON data, we create nodes (Drug, Disease, AdminRoute, ApprovalYear) and relationships such as `HAS_INDICATION`, `HAS_CONTRAINDICATION`, `HAS_ADMIN_ROUTE`, and `HAS_APPROVAL_YEAR`.

Identifiers come from [DrugBank](https://go.drugbank.com/), [Chemical Entities of Biological Interest](https://www.ebi.ac.uk/chebi/) (ChEBI), [Disease Ontology](https://disease-ontology.org/) (DO), and the [Orphanet](https://www.orpha.net/).

This step uses the official Python Neo4j driver to insert the entities and enforce uniqueness constraints, enabling efficient queries across the graph.


⚠️ **IMPORTANT**:

You need to create a Neo4j account first. Follow these steps:
1. Sign up for Neo4j with your credentials.
2. Create a free AuraDB instance.
3. After selecting that option, you will see the credentials for `Instance01`. Click `Download and Continue`.
4. Open the downloaded `.txt` file and copy the values for `NEO4J_URI`, `NEO4J_USERNAME`, and `NEO4J_PASSWORD` to use in step 3.2.


<a id="3.1"></a>
### 3.1. Import Libraries


In [ ]:
!pip install neo4j

In [ ]:
import json
import os
from neo4j import GraphDatabase
from datetime import datetime

<a id="3.2"></a>
### 3.2. Enter your Neo4j credentials

In [ ]:
# Section 2: Use your Neo4j credentials
uri = "bolt://localhost:7687"      # Neo4j server address
user = "neo4j"                     # Neo4j username
password = "your_password"         # Neo4j passowrd

<a id="3.3"></a>
### 3.3. Neo4j Handler class

In [ ]:
class Neo4jHandler:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    def close(self):
        self.driver.close()
    def create_constraints(self):
        with self.driver.session() as session:
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Drug) REQUIRE d.id IS UNIQUE")
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (disease:Disease) REQUIRE disease.id IS UNIQUE")
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (r:AdminRoute) REQUIRE r.name IS UNIQUE")
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (y:ApprovalYear) REQUIRE y.year IS UNIQUE")
    def create_drug(self, drug):
        query = """
        MERGE (d:Drug {id: $id})
        SET d.name = $name, d.organization = $organization, d.effectiveTime = $effectiveTime
        """
        with self.driver.session() as session:
            session.run(query, **drug)
    def create_disease(self, disease):
        query = """
        MERGE (d:Disease {id: $id})
        SET d.name = $name
        """
        with self.driver.session() as session:
            session.run(query, **disease)
    def create_admin_route(self, route, drug_id):
        if not route:
            return
        routes = [r.strip() for r in route.split(",")]
        with self.driver.session() as session:
            for r in routes:
                query = """
                MERGE (r:AdminRoute {name: $route})
                WITH r
                MATCH (d:Drug {id: $drug_id})
                MERGE (d)-[:ADMINISTERED_VIA]->(r)
                """
                session.run(query, route=r, drug_id=drug_id)
    def create_approval_year(self, year, drug_id):
        if not year:
            return
        query = """
        MERGE (y:ApprovalYear {year: $year})
        WITH y
        MATCH (d:Drug {id: $drug_id})
        MERGE (d)-[:APPROVED_IN]->(y)
        """
        with self.driver.session() as session:
            session.run(query, year=year, drug_id=drug_id)
    def create_relationships(self, drug_id, indications, contraindications, route, approval_year):
        with self.driver.session() as session:
            for ind in indications:
                session.run("""
                MATCH (d:Drug {id: $drug_id}), (dis:Disease {id: $dis_id})
                MERGE (d)-[:TREATS]->(dis)
                """, {"drug_id": drug_id, "dis_id": ind["id"]})
            for con in contraindications:
                session.run("""
                MATCH (d:Drug {id: $drug_id}), (dis:Disease {id: $dis_id})
                MERGE (d)-[:CONTRAINDICATED_FOR]->(dis)
                """, {"drug_id": drug_id, "dis_id": con["id"]})
            if route:
                self.create_admin_route(route, drug_id)
            if approval_year:
                self.create_approval_year(approval_year, drug_id)
    def insert_data(self, drug, diseases, indications, contraindications):
        self.create_drug(drug)
        for disease in diseases:
            self.create_disease(disease)
        self.create_admin_route(drug.get("admin_route"), drug["id"])
        self.create_approval_year(drug.get("approval_year"), drug["id"])
        self.create_relationships(drug["id"], indications, contraindications, drug.get("admin_route"), drug.get("approval_year"))

<a id="3.4"></a>
### 3.4. Utility functions for data processing

In [ ]:
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

def clean_id(url):
    if not url or not isinstance(url, str):
        return None
    if url.startswith('http://purl.obolibrary.org/obo/'):
        return url.split('/')[-1]
    return url.split('/')[-1]

def extract_year(approval_date):
    if not approval_date:
        return None
    if isinstance(approval_date, int):
        return str(approval_date)[:4]
    if isinstance(approval_date, str):
        try:
            parsed_date = datetime.strptime(approval_date, "%b %d, %Y")
            return str(parsed_date.year)
        except ValueError:
            try:
                parsed_date = datetime.strptime(approval_date, "%B %d, %Y")
                return str(parsed_date.year)
            except ValueError:
                pass
        if approval_date.isdigit():
            if len(approval_date) == 8:
                return approval_date[:4]
            elif len(approval_date) == 4:
                return approval_date
    return None

def process_xml_file(file_path, neo4j_handler):
    try:
        data = load_json(file_path)
        if "ingredients" in data:
            if not data.get("ingredients"):
                return
            main_ingredient = data["ingredients"][0]
            chebi_id = clean_id(main_ingredient.get("chebi_id"))
            drugbank_id = clean_id(main_ingredient.get("drugbank_id"))
            drug_id = chebi_id or drugbank_id
            if not drug_id:
                return
            drug = {
                "id": drug_id,
                "name": main_ingredient.get("name"),
                "organization": data.get("organization"),
                "effectiveTime": data.get("effectiveTime"),
                "admin_route": data.get("admin_route"),
                "approval_year": extract_year(data.get("approval_date")),
                "chebi_id": chebi_id,
                "drugbank_id": drugbank_id
            }
            diseases = []
            indications = []
            contraindications = []
            if "indications" in data and isinstance(data["indications"], dict):
                for entity in data["indications"].get("doid_entities", []):
                    if isinstance(entity, dict):
                        disease_id = clean_id(entity.get("doid_id"))
                        if disease_id:
                            disease_name = entity.get("text", "Unknown")
                            if " (DOID:" in disease_name:
                                disease_name = disease_name.split(" (DOID:")[0]
                            diseases.append({"id": disease_id, "name": disease_name})
                            indications.append({"id": disease_id})
                for entity in data["indications"].get("orphanet_entities", []):
                    if isinstance(entity, dict):
                        disease_id = clean_id(entity.get("id"))
                        if disease_id:
                            disease_name = entity.get("text", "Unknown")
                            diseases.append({"id": disease_id, "name": disease_name})
                            indications.append({"id": disease_id})
            if "contraindications" in data and isinstance(data["contraindications"], dict):
                for entity in data["contraindications"].get("doid_entities", []):
                    if isinstance(entity, dict):
                        disease_id = clean_id(entity.get("doid_id"))
                        if disease_id:
                            disease_name = entity.get("text", "Unknown")
                            if " (DOID:" in disease_name:
                                disease_name = disease_name.split(" (DOID:")[0]
                            diseases.append({"id": disease_id, "name": disease_name})
                            contraindications.append({"id": disease_id})
                for entity in data["contraindications"].get("orphanet_entities", []):
                    if isinstance(entity, dict):
                        disease_id = clean_id(entity.get("id"))
                        if disease_id:
                            disease_name = entity.get("text", "Unknown")
                            diseases.append({"id": disease_id, "name": disease_name})
                            contraindications.append({"id": disease_id})
            print(f"Drug: {drug_id} (CHEBI: {chebi_id}, DrugBank: {drugbank_id})")
            print(f"Admin Route: {drug.get('admin_route')}, Approval Year: {drug.get('approval_year')}")
            print(f"Found {len(diseases)} diseases, {len(indications)} indications, {len(contraindications)} contraindications")
            neo4j_handler.insert_data(drug, diseases, indications, contraindications)
        else:
            for drug_entry in data.get("drug", []):
                chebi_id = clean_id(drug_entry.get("chebi_id"))
                drugbank_id = clean_id(drug_entry.get("drugbank_id"))
                drug_id = chebi_id or drugbank_id
                if not drug_id:
                    continue
                approval_date = (
                    data.get("Approval_Date") or
                    data.get("Approval Date") or
                    drug_entry.get("approval_date") or
                    data.get("effectiveTime") or
                    data.get("date")
                )
                approval_year = extract_year(approval_date)
                admin_route = drug_entry.get("admin_route") or data.get("Route of Administration")
                drug = {
                    "id": drug_id,
                    "name": drug_entry.get("name"),
                    "organization": data.get("organization") or data.get("Applicant"),
                    "effectiveTime": data.get("effectiveTime"),
                    "admin_route": admin_route,
                    "approval_year": approval_year,
                    "chebi_id": chebi_id,
                    "drugbank_id": drugbank_id
                }
                diseases = []
                indications = []
                contraindications = []
                for ind in data.get("indications", []):
                    doid_id = clean_id(ind.get("doid_id"))
                    orphanet_id = clean_id(ind.get("orphanet_id"))
                    disease_id = doid_id or orphanet_id
                    if disease_id:
                        diseases.append({"id": disease_id, "name": ind.get("text", "Unknown")})
                        indications.append({"id": disease_id})
                for con in data.get("contraindications", []):
                    doid_id = clean_id(con.get("doid_id"))
                    orphanet_id = clean_id(con.get("orphanet_id"))
                    disease_id = doid_id or orphanet_id
                    if disease_id:
                        diseases.append({"id": disease_id, "name": con.get("text", "Unknown")})
                        contraindications.append({"id": disease_id})
                print(f"Drug: {drug_id} (CHEBI: {chebi_id}, DrugBank: {drugbank_id})")
                print(f"Admin Route: {admin_route}, Approval Year: {approval_year}")
                print(f"Found {len(diseases)} diseases, {len(indications)} indications, {len(contraindications)} contraindications")
                neo4j_handler.insert_data(drug, diseases, indications, contraindications)
    except Exception as e:
        print(f"Error processing file '{file_path}': {e}")

<a id="3.5"></a>
### 3.5. Establish connection to Neo4j


In [ ]:
# Section 4: Create a Neo4j handler and define constraints
neo4j_handler = Neo4jHandler(uri, user, password)
neo4j_handler.create_constraints()
print("Established connection and created constraints.")

<a id="3.6"></a>
### 3.6. Import data from JSON files


In [ ]:
# Path to the project's JSON folder
entities_path = BASE_DIR / 'entities'

# List all contents of the folder recursively
print(f"Listing all contents of {entities_path} (recursively):")
for path in entities_path.rglob('*'):
    print(path)

# Recursively search for JSON files within the project
json_files = list(entities_path.rglob('*.json'))

if not json_files:
    print(f"No JSON files found in {entities_path}")
else:
    print(f"Found {len(json_files)} JSON files:")
    for json_file in json_files:
        print(f"  - {json_file}")

    # Import function (make sure process_xml_file and neo4j_handler are defined)
    def import_data(file_path):
        process_xml_file(file_path, neo4j_handler)
        print(f"Import completed for file: {file_path}")

    # Process all found JSON files
    for json_file in json_files:
        import_data(str(json_file))


<a id="3.7"></a>
### 3.7. Close Neo4j connection


In [ ]:
# Close Neo4j connection
neo4j_handler.close()
print("Neo4j connection closed")